In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models-transformers-net-item-prediction/nvidia_logo.png" style="width: 90px; float: right;">

# Transformer-based architecture for next-item prediction task

## Overview

In this use case we will train a Transformer-based architecture for next-item prediction task.

**Note, the data for this notebook will be automatically downloaded to the folder specified in the cells below.**

We will use the [booking.com dataset](https://github.com/bookingcom/ml-dataset-mdt) to train a session-based model. The dataset contains 1,166,835 of anonymized hotel reservations in the train set and 378,667 in the test set. Each reservation is a part of a customer's trip (identified by `utrip_id`) which includes consecutive reservations.

We will reshape the data to organize it into 'sessions'. Each session will be a full customer itinerary in chronological order. The goal will be to predict the city_id of the final reservation of each trip.


### Learning objectives

- Training a Transformer-based architecture for next-item prediction task

## Downloading and preparing the dataset

We will download the dataset using a functionality provided by merlin models. The dataset can be found on GitHub [here](https://github.com/bookingcom/ml-dataset-mdt).

**Read more about libraries used in the import statements below**

- [get_lib](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/core/dispatch.py)
- [get_booking](https://github.com/NVIDIA-Merlin/models/tree/main/merlin/datasets/ecommerce)
- [nvtabular](https://github.com/NVIDIA-Merlin/NVTabular/tree/main/nvtabular)
- [nvtabular ops](https://github.com/NVIDIA-Merlin/NVTabular/tree/main/nvtabular/ops)
- [schema tags](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/tags.py)
- [merlin models tensorflow](https://github.com/NVIDIA-Merlin/models/tree/main/merlin/models/tf)
- [get_booking](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/datasets/ecommerce/booking/dataset.py)

In [2]:
# Resetting the TF memory allocation to not be 50% by default. 
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

from merlin.core.dispatch import get_lib
from merlin.datasets.ecommerce import get_booking

import numpy as np

from nvtabular import *
from nvtabular import ops

from merlin.schema.tags import Tags
import merlin.models.tf as mm

get_booking('/workspace/data')
train = get_lib().read_csv('/workspace/data/train_set.csv', parse_dates=['checkin', 'checkout'])

print('Training data import complete')

2023-02-08 13:16:58.296917: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-02-08 13:17:00.184118: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 13:17:00.184683: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 13:17:00.184852: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 13:17:00.528889: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate 

Training data import complete


/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.SESSION_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.SESSION: 'session'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Each reservation has a unique `utrip_id`. During each trip a customer vists several destinations.

In [3]:
# When displaying cudf dataframes use print() or display(), otherwise Jupyter creates hidden copies.
print(train.head())

   user_id    checkin   checkout  city_id device_class  affiliate_id  \
0  1000027 2016-08-13 2016-08-14     8183      desktop          7168   
1  1000027 2016-08-14 2016-08-16    15626      desktop          7168   
2  1000027 2016-08-16 2016-08-18    60902      desktop          7168   
3  1000027 2016-08-18 2016-08-21    30628      desktop           253   
4  1000033 2016-04-09 2016-04-11    38677       mobile           359   

  booker_country hotel_country   utrip_id  
0        Elbonia        Gondal  1000027_1  
1        Elbonia        Gondal  1000027_1  
2        Elbonia        Gondal  1000027_1  
3        Elbonia        Gondal  1000027_1  
4         Gondal  Cobra Island  1000033_1  


We will train on sequences of `city_id` and `booker_country` and based on this information, our model will attempt to predict the next `city_id` (the next hop in the journey).

We will train a transformer model that can work with sequences of variable length within a batch. This functionality is provided to us out of the box and doesn't require any changes to the architecture. Thanks to it we do not have to pad or trim our sequences to any particular length -- our model can make effective use of all of the data!

*With one exception.* For a masked language model that we will be training, we need to discard sequences that are shorter than two hops. This makes sense as there is nothing our model could learn if it was only presented with an itinerary with a single destination on it!

Let us begin by splitting the data into a train and validation set based on trip ID.

Let's see how many unique trips there are in the dataset. Also, let us shuffle the trips along the way so that our validation set consists of a random sample of our train set.

In [4]:
# Unique trip ids.
utrip_ids = train.sample(frac=1).utrip_id.unique()
print('Number of unique trips is :', len(utrip_ids))

Number of unique trips is : 217686


Now let's assign data to our train and validation sets. Furthermore, we sort the data by `utrip_id` and `checkin`. This way we ensure our sequences of visited `city_ids` will be in proper order!

In [5]:
train_set_utrip_ids = utrip_ids[:int(0.8 * utrip_ids.shape[0])]
validation_set_utrip_ids = utrip_ids[int(0.8 * utrip_ids.shape[0]):]

train_set = train[train.utrip_id.isin(train_set_utrip_ids)].sort_values(['utrip_id', 'checkin'])
validation_set = train[train.utrip_id.isin(validation_set_utrip_ids)].sort_values(['utrip_id', 'checkin'])

##  Preprocessing with NVTabular

We can now begin with data preprocessing.

We will combine trips into "sessions", discard trips that are too short and calculate total trip length.

We will use NVTabular for this work. It offers optimized tabular data preprocessing operators that run on the GPU. If you would like to learn more about the NVTabular library, please take a look [here](https://github.com/NVIDIA-Merlin/NVTabular).

Read more about the [Merlin's Dataset API](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/io/dataset.py)  
Read more about how [parquet files are read in and processed by Merlin](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/io/parquet.py)  
Read more about [Tags](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/tags.py)  
- [schema_select_by_tag](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/schema.py)  

Read more about [NVTabular Workflows](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/workflow/workflow.py)  
- [fit_transform](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/workflow/workflow.py)
- [transform](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/workflow/workflow.py)  

Read more about the [NVTabular Operators]()  
- [Categorify](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/categorify.py)
- [AddTags](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/add_metadata.py)
- [LambdaOp](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/lambdaop.py)
- [Rename](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/rename.py)
- [Filter](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/filter.py)



In [6]:
train_set_dataset = Dataset(train_set)
validation_set_dataset = Dataset(validation_set)

In [7]:
weekday_checkin = (
    ["checkin"]
    >> ops.LambdaOp(lambda col: col.dt.weekday)
    >> ops.Rename(name="weekday_checkin")
)

weekday_checkout = (
    ["checkout"]
    >> ops.LambdaOp(lambda col: col.dt.weekday)
    >> ops.Rename(name="weekday_checkout")
)

categorical_features = (['city_id', 'booker_country', 'hotel_country'] +
                         weekday_checkin + weekday_checkout
                       ) >> ops.Categorify(start_index=1)  

groupby_features = categorical_features + ['utrip_id', 'checkin'] >> ops.Groupby(
    groupby_cols=['utrip_id'],
    aggs={
        'city_id': ['list', 'count'],
        'booker_country': ['list'],
        'hotel_country': ['list'],
        'weekday_checkin': ['list'],
        'weekday_checkout': ['list']
    },
    sort_cols="checkin"
)

list_features = (
            groupby_features['city_id_list', 'booker_country_list', 'hotel_country_list', 
                 'weekday_checkin_list', 'weekday_checkout_list'
            ] >> ops.AddTags([Tags.SEQUENCE])
)

# Filter out sessions with less than 2 interactions 
MINIMUM_SESSION_LENGTH = 2
features = list_features + (groupby_features['city_id_count'] >>  ops.AddTags([Tags.CONTINUOUS]))
filtered_sessions = features >> ops.Filter(f=lambda df: df["city_id_count"] >= MINIMUM_SESSION_LENGTH) 

In [8]:
wf = Workflow(filtered_sessions)

In [9]:
train_set_processed = wf.fit_transform(train_set_dataset)
validation_set_processed = wf.transform(validation_set_dataset)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Our data consists of a sequence of visited `city_ids`, a sequence of `booker_countries` (represented as integer categories) and a `city_id_count` column (which contains the count of visited cities in a trip).

In [10]:
train_set_processed.compute().head()

city_id_list    booker_country_list  \
0      [8240, 156, 2278, 2097]           [3, 3, 3, 3]   
1      [63, 1160, 87, 619, 63]        [1, 1, 1, 1, 1]   
2  [7, 6, 24, 1051, 65, 52, 3]  [2, 2, 2, 2, 2, 2, 2]   
3          [1032, 757, 140, 3]           [2, 2, 2, 2]   
4   [3603, 262, 662, 250, 359]        [1, 1, 1, 1, 1]   

        hotel_country_list   weekday_checkin_list  weekday_checkout_list  \
0             [3, 3, 3, 3]           [5, 7, 4, 3]           [7, 4, 2, 7]   
1          [1, 1, 1, 1, 1]        [5, 1, 4, 3, 5]        [6, 4, 2, 5, 4]   
2  [2, 2, 2, 16, 16, 3, 3]  [5, 1, 2, 6, 5, 7, 4]  [6, 3, 1, 5, 7, 4, 3]   
3          [19, 19, 19, 3]           [1, 4, 2, 3]           [4, 3, 2, 5]   
4     [30, 30, 30, 30, 30]        [1, 3, 6, 5, 1]        [2, 1, 5, 6, 3]   

   city_id_count  
0              4  
1              5  
2              7  
3              4  
4              5

We are now ready to train our model.

Here is the schema of the data that our model will use.

In [11]:
seq_schema = train_set_processed.schema.select_by_tag(Tags.SEQUENCE)
seq_schema

[{'name': 'city_id_list', 'tags': {<Tags.SEQUENCE: 'sequence'>, <Tags.CATEGORICAL: 'categorical'>, <Tags.ITEM_ID: 'item_id'>, <Tags.ITEM: 'item'>, <Tags.ID: 'id'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.city_id_list.parquet', 'domain': {'min': 0, 'max': 37202, 'name': 'city_id_list'}, 'embedding_sizes': {'cardinality': 37203, 'dimension': 512}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}, {'name': 'booker_country_list', 'tags': {<Tags.CATEGORICAL: 'categorical'>, <Tags.SEQUENCE: 'sequence'>, <Tags.ITEM: 'item'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.booker_country_list.parquet', 'domain': {'min': 0, 'max': 5, 'name': 'booker_country_list'}, 'embedding_sizes': {'cardinality': 6, 'dimension': 16}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}, {'name': 'hotel_country_list', 'tags': {<Tags.CATEGORICAL: 'categorical'>, <Tags.SEQUENCE: 'sequence'>, <Tags.ITEM: 'item'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.hotel_country_list.parquet', 'domain': {'min': 0, 'max': 194, 'name': 'hotel_country_list'}, 'embedding_sizes': {'cardinality': 195, 'dimension': 31}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}, {'name': 'weekday_checkin_list', 'tags': {<Tags.CATEGORICAL: 'categorical'>, <Tags.SEQUENCE: 'sequence'>, <Tags.ITEM: 'item'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.weekday_checkin_list.parquet', 'domain': {'min': 0, 'max': 7, 'name': 'weekday_checkin_list'}, 'embedding_sizes': {'cardinality': 8, 'dimension': 16}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}, {'name': 'weekday_checkout_list', 'tags': {<Tags.CATEGORICAL: 'categorical'>, <Tags.SEQUENCE: 'sequence'>, <Tags.ITEM: 'item'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.weekday_checkout_list.parquet', 'domain': {'min': 0, 'max': 7, 'name': 'weekday_checkout_list'}, 'embedding_sizes': {'cardinality': 8, 'dimension': 16}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}]

Align the schema of train and validation datasets with the model's schema

In [ ]:
train_set_processed.schema = seq_schema
validation_set_processed.schema = seq_schema

Let's also identify the target column.

In [12]:
target = train_set_processed.schema.select_by_tag(Tags.SEQUENCE).column_names[0]
target

'city_id_list'

## Constructing the model

Let's construct our model.

We can specify various hyperparameters, such as the number of heads and number of layers to use.

For the transformer portion of our model, we will use the `XLNet` architecture.

Later, when we run the `fit` method on our model, we will specify the `masking_probability` of `0.3` and link it to the transformer block defined in out model. Through the combination of these parameters, our model will train on sequences where any given timestep will be masked with a probability of 0.3 and it will be our model's training task to infer the target value for that step!

To summarize, Masked Language Modeling is implemented by:

* `SequenceMaskRandom()` - Used as a pre for model.fit(), it randomly selects items from the sequence to be masked for prediction as targets, by using Keras masking. This block also adds the necessary configuration to the specified `transformer` block so as it
is pre-configured with the necessary layers needed to prepare the inputs to the HuggingFace transformer layer and to post-process its outputs. For example, one pre-processing operation is to replace the input embeddings at masked positions for prediction by a dummy trainable embedding, to avoid leakage of the targets.


**Read more about the apis used to construct models** 
- [blocks](https://github.com/NVIDIA-Merlin/models/tree/main/merlin/models/tf/blocks)
- [MLPBlock](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/blocks/mlp.py)
- [InputBlockV2](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/inputs/base.py)
- [Embeddings](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/inputs/embedding.py)
- [XLNetBlock](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/transformers/block.py)
- [CategoricalOutput](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/outputs/classification.py)
- [.schema.select_by_name](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/schema.py)
- [.schema.select_by_tag](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/schema.py)
- [model.compile()](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/models/base.py)
- [model.fit()](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/models/base.py)
- [model.evaluate()](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/models/base.py)
- [mm.SequenceMaskRandom](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/transforms/sequence.py)
- [mm.SequenceMaskLast](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/transforms/sequence.py)

In [13]:
dmodel=48
mlp_block = mm.MLPBlock(
                [128,dmodel],
                activation='relu',
                no_activation_last_layer=True,
            )
transformer_block = mm.XLNetBlock(d_model=dmodel, n_head=4, n_layer=2)
model = mm.Model(
    mm.InputBlockV2(
        seq_schema,
        embeddings=mm.Embeddings(
            train_set_processed.schema.select_by_tag(Tags.CATEGORICAL), sequence_combiner=None
        ),
    ),
    mlp_block,
    transformer_block,
    mm.CategoricalOutput(
        train_set_processed.schema.select_by_name(target),
        default_loss="categorical_crossentropy",
    ),
)

## Model training

In [14]:
model.compile(run_eagerly=False, optimizer='adam', loss="categorical_crossentropy")
model.fit(train_set_processed, batch_size=64, epochs=5, pre=mm.SequenceMaskRandom(schema=seq_schema, target=target, masking_prob=0.3, transformer=transformer_block))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
2023-02-08 13:17:18.083919: I tensorflow/stream_executor/cuda/cuda_blas.cc:1633] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-08 13:17:18.254522: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700


Epoch 1/5


/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 48), dtype=float32), dense_shape=Tensor("gradient_tape/model/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/xl_net_block/replace_masked_embeddings/RaggedWhere/Reshape_3:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/model/xl_ne

2023-02-08 13:17:26.931275: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model/xl_net_block/replace_masked_embeddings/RaggedWhere/Assert/AssertGuard/branch_executed/_99


2720/2720 [==============================] - 112s 35ms/step - loss: 1.0077 - recall_at_10: 0.0725 - mrr_at_10: 0.0307 - ndcg_at_10: 0.0405 - map_at_10: 0.0307 - precision_at_10: 0.0073 - regularization_loss: 0.0000e+00 - loss_batch: 1.0077
Epoch 2/5
2720/2720 [==============================] - 104s 35ms/step - loss: 0.8326 - recall_at_10: 0.1148 - mrr_at_10: 0.0518 - ndcg_at_10: 0.0666 - map_at_10: 0.0518 - precision_at_10: 0.0115 - regularization_loss: 0.0000e+00 - loss_batch: 0.8327
Epoch 3/5
2720/2720 [==============================] - 104s 35ms/step - loss: 0.7727 - recall_at_10: 0.1392 - mrr_at_10: 0.0638 - ndcg_at_10: 0.0816 - map_at_10: 0.0638 - precision_at_10: 0.0139 - regularization_loss: 0.0000e+00 - loss_batch: 0.7728
Epoch 4/5
2720/2720 [==============================] - 106s 36ms/step - loss: 0.7374 - recall_at_10: 0.1575 - mrr_at_10: 0.0734 - ndcg_at_10: 0.0932 - map_at_10: 0.0734 - precision_at_10: 0.0157 - regularization_loss: 0.0000e+00 - loss_batch: 0.7374
Epoch 5/5


## Model evaluation

We have trained our model.

But in training the metrics come from a masked language modelling task. A portion of steps in the sequence was masked for each example. The metrics were calculated on this task.

In reality, we probably care how well our model does on the next item prediction task (as it mimics the scenario in which the model would be likely to be used).

Let's measure the performance of the model on a task where it attempts to predict the last item in a sequence.

We will mask the last item using `SequenceMaskLast` and run inference.

In [15]:
model.evaluate(
    validation_set_processed,
    batch_size=128,
    pre=mm.SequenceMaskLast(schema=validation_set_processed.schema, target=target, transformer=transformer_block),
    return_dict=True
)

2023-02-08 13:26:13.229779: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model/xl_net_block/replace_masked_embeddings/RaggedWhere/Assert/AssertGuard/branch_executed/_74


340/340 [==============================] - 19s 44ms/step - loss: 0.3306 - recall_at_10: 0.5549 - mrr_at_10: 0.3080 - ndcg_at_10: 0.3668 - map_at_10: 0.3080 - precision_at_10: 0.0555 - regularization_loss: 0.0000e+00 - loss_batch: 0.3304


{'loss': 0.3305854797363281,
 'recall_at_10': 0.5549300312995911,
 'mrr_at_10': 0.3080165982246399,
 'ndcg_at_10': 0.36679476499557495,
 'map_at_10': 0.3080165982246399,
 'precision_at_10': 0.05549299716949463,
 'regularization_loss': 0.0,
 'loss_batch': 0.28271856904029846}

## Summary

We have trained a transformer model for the next item prediction task using language model masking.

For another session-based example that goes deeper into data preprocessing and that covers several advanced techniques (Weight Tying, Temperature Scaling) please see [Session-Based Next Item Prediction for Fashion E-Commerce](https://github.com/NVIDIA-Merlin/models/blob/t4rec_use_case/examples/usecases/ecommerce-session-based-next-item-prediction-for-fashion.ipynb). 